In [1]:
import pandas as pd
import numpy as np


In [5]:
# Load raw dataset
df = pd.read_csv("/content/global_superstore2.csv", encoding="latin1")

# Quick inspection
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,...,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
0,32298,CA-2012-124891,31-07-2012,31-07-2012,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,...,TEC-AC-10003033,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,2309.650,7,0.0,762.1845,933.57,Critical
1,26341,IN-2013-77878,05-02-2013,07-02-2013,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,...,FUR-CH-10003950,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.63,Critical
2,25330,IN-2013-71249,17-10-2013,18-10-2013,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,...,TEC-PH-10004664,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.49,Medium
3,13524,ES-2013-1579342,28-01-2013,30-01-2013,First Class,KM-16375,Katherine Murray,Home Office,Berlin,Berlin,...,TEC-PH-10004583,Technology,Phones,"Motorola Smart Phone, Cordless",2892.510,5,0.1,-96.5400,910.16,Medium
4,47221,SG-2013-4320,05-11-2013,06-11-2013,Same Day,RH-9495,Rick Hansen,Consumer,Dakar,Dakar,...,TEC-SHA-10000501,Technology,Copiers,"Sharp Wireless Fax, High-Speed",2832.960,8,0.0,311.5200,903.04,Critical


In [6]:
# Dataset shape
print("Rows, Columns:", df.shape)

# Column names
df.columns


Rows, Columns: (51290, 24)


Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'City', 'State', 'Country',
       'Postal Code', 'Market', 'Region', 'Product ID', 'Category',
       'Sub-Category', 'Product Name', 'Sales', 'Quantity', 'Discount',
       'Profit', 'Shipping Cost', 'Order Priority'],
      dtype='object')

In [7]:
df.columns = (
    df.columns
    .str.strip()
    .str.replace(" ", "_")
    .str.replace("-", "_")
)


In [8]:
df.columns


Index(['Row_ID', 'Order_ID', 'Order_Date', 'Ship_Date', 'Ship_Mode',
       'Customer_ID', 'Customer_Name', 'Segment', 'City', 'State', 'Country',
       'Postal_Code', 'Market', 'Region', 'Product_ID', 'Category',
       'Sub_Category', 'Product_Name', 'Sales', 'Quantity', 'Discount',
       'Profit', 'Shipping_Cost', 'Order_Priority'],
      dtype='object')

In [9]:
# Missing values
df.isnull().sum().sort_values(ascending=False)


,0
Postal_Code,41296
Row_ID,0
Order_Date,0
Order_ID,0
Ship_Mode,0
Customer_ID,0
Customer_Name,0
Ship_Date,0
Segment,0
City,0


In [10]:
# Duplicate rows
df.duplicated().sum()


np.int64(0)

In [11]:
# Convert date columns
df['Order_Date'] = pd.to_datetime(df['Order_Date'])
df['Ship_Date'] = pd.to_datetime(df['Ship_Date'])

# Remove duplicates
df = df.drop_duplicates()

# Drop rows with critical missing values
df = df.dropna(subset=['Sales', 'Profit', 'Order_Date'])


/tmp/ipython-input-256044613.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Order_Date'] = pd.to_datetime(df['Order_Date'])
/tmp/ipython-input-256044613.py:3: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Ship_Date'] = pd.to_datetime(df['Ship_Date'])


**Time-Based Feature Engineering**

In [12]:
df['Order_Year'] = df['Order_Date'].dt.year
df['Order_Month'] = df['Order_Date'].dt.month
df['Order_Month_Name'] = df['Order_Date'].dt.month_name()
df['Order_Quarter'] = df['Order_Date'].dt.to_period('Q').astype(str)


**Profitability Metrics**

In [13]:
# Profit margin %
df['Profit_Margin_Pct'] = (df['Profit'] / df['Sales']) * 100

# Profit flag
df['Profit_Flag'] = np.where(df['Profit'] > 0, 'Profitable', 'Loss')


**Order Value Segmentation (Business Intelligence)**

This enables 80/20 analysis and high-value customer insights.

In [14]:
# Quartile thresholds
q1 = df['Sales'].quantile(0.25)
q3 = df['Sales'].quantile(0.75)

def order_segment(sales):
    if sales >= q3:
        return 'High Value'
    elif sales <= q1:
        return 'Low Value'
    else:
        return 'Medium Value'

df['Order_Value_Segment'] = df['Sales'].apply(order_segment)


**Operational Metrics (Hidden Differentiator)**

In [15]:
# Shipping delay in days
df['Shipping_Delay_Days'] = (df['Ship_Date'] - df['Order_Date']).dt.days

# Delay flag
df['Delayed_Shipment'] = np.where(df['Shipping_Delay_Days'] > 5, 'Yes', 'No')


**Final Sanity Checks**

In [16]:
df.describe(include='all').transpose()


,count,unique,top,freq,mean,min,25%,50%,75%,max,std
Row_ID,51290.0,NaN,NaN,NaN,25645.5,1.0,12823.25,25645.5,38467.75,51290.0,14806.29199
Order_ID,51290,25035,CA-2014-100111,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Order_Date,51290,NaN,NaN,NaN,2013-05-11 21:26:49.155781120,2011-01-01 00:00:00,2012-06-19 00:00:00,2013-07-08 00:00:00,2014-05-22 00:00:00,2014-12-31 00:00:00,NaN
Ship_Date,51290,NaN,NaN,NaN,2013-05-15 20:42:42.745174528,2011-01-03 00:00:00,2012-06-23 00:00:00,2013-07-12 00:00:00,2014-05-26 00:00:00,2015-01-07 00:00:00,NaN
Ship_Mode,51290,4,Standard Class,30775,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Customer_ID,51290,1590,PO-18850,97,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Customer_Name,51290,795,Muhammed Yedwab,108,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Segment,51290,3,Consumer,26518,NaN,NaN,NaN,NaN,NaN,NaN,NaN
City,51290,3636,New York City,915,NaN,NaN,NaN,NaN,NaN,NaN,NaN
State,51290,1094,California,2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
df.head()

,Row_ID,Order_ID,Order_Date,Ship_Date,Ship_Mode,Customer_ID,Customer_Name,Segment,City,State,...,Order_Priority,Order_Year,Order_Month,Order_Month_Name,Order_Quarter,Profit_Margin_Pct,Profit_Flag,Order_Value_Segment,Shipping_Delay_Days,Delayed_Shipment
0,32298,CA-2012-124891,2012-07-31,2012-07-31,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,...,Critical,2012,7,July,2012Q3,33.000000,Profitable,High Value,0,No
1,26341,IN-2013-77878,2013-02-05,2013-02-07,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,...,Critical,2013,2,February,2013Q1,-7.784693,Loss,High Value,2,No
2,25330,IN-2013-71249,2013-10-17,2013-10-18,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,...,Medium,2013,10,October,2013Q4,17.776630,Profitable,High Value,1,No
3,13524,ES-2013-1579342,2013-01-28,2013-01-30,First Class,KM-16375,Katherine Murray,Home Office,Berlin,Berlin,...,Medium,2013,1,January,2013Q1,-3.337586,Loss,High Value,2,No
4,47221,SG-2013-4320,2013-11-05,2013-11-06,Same Day,RH-9495,Rick Hansen,Consumer,Dakar,Dakar,...,Critical,2013,11,November,2013Q4,10.996272,Profitable,High Value,1,No


 **Export BI-READY DATASET**

In [18]:
df.to_csv("/content/cleaned_global_superstore2.csv", index=False)

print("Cleaned dataset exported successfully.")


Cleaned dataset exported successfully.
